In [4]:
import pandas
import math
import numpy as np

def get_res(DATASET, SKEW, PARTIES):
    experiment_directory = f"{DATASET}_non-IID_feature_skew/{PARTIES}_parties/"

    weighted_mean = {'client_lr':0.0,
                    'client_momentum':0.0,
                    'batch_size':0}
    
    lr = np.zeros(PARTIES, dtype=float)
    momentum = np.zeros(PARTIES, dtype=float)
    batch_size = np.zeros(PARTIES, dtype=float)
    accuracy = np.zeros(PARTIES, dtype=float)

    # Get individual data
    for i in range(PARTIES):

        client_result = ""

        try:
            client_result = pandas.read_csv(f"{experiment_directory}individual/{SKEW}/{SKEW}_feature_{PARTIES}clts_clt{i}.txt")
            #display(client_result.head(1).get(['val_accuracy','client_lr','client_momentum','round_epochs','batch_size']))
            
            client_lr = client_result.head(1).get(['client_lr']).values[0][0]
            client_momentum = client_result.head(1).get(['client_momentum']).values[0][0]
            client_batch_size = client_result.head(1).get(['batch_size']).values[0][0]
            client_accuracy = client_result.head(1).get(['val_accuracy']).values[0][0]
            
            lr[i] = client_lr
            momentum[i] = client_momentum
            batch_size[i] = client_batch_size
            accuracy[i] = client_accuracy
                        
        except:
            print(f"File does not exist, client {i} crashed.")
    
    best_acc = np.max(accuracy)
    
    for i in range(PARTIES):
        #################  AGGREGATION METHOD BEGIN  #################
        weighted_mean['client_lr'] += lr[i] * (1-lr[i]) * (1-abs(best_acc-accuracy[i])) / (10*PARTIES)
        weighted_mean['client_momentum'] += min(1.0/PARTIES , momentum[i] * (1-momentum[i]) * (1-abs(best_acc-accuracy[i]))*10/PARTIES)
        weighted_mean['batch_size'] += batch_size[i] * accuracy[i] / PARTIES
        #weighted_mean['client_lr'] += lr[i] * (1-lr[i]) * (1-abs(best_acc-accuracy[i])) / 100
        #weighted_mean['client_momentum'] += min(1.0/PARTIES , momentum[i] * (1-momentum[i]) * (1-abs(best_acc-accuracy[i])))
        #weighted_mean['batch_size'] += batch_size[i] * accuracy[i] / 10
        #################  AGGREGATION METHOD END  #################
        
    # Get fedavg data
    try:
        with open(f"{experiment_directory}fedavg/{DATASET.lower()}_feature_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("Federated average best parameters\n")

            for i in range(1):
                print(reader.readline())

    except:
        print("File does not exist")

    print(weighted_mean)
    print("\nDecentralized intervals search\n")
    # Get decentralized intervals search
    try:
        with open(f"{experiment_directory}individual/{SKEW}/{DATASET.lower()}_feature_skew_{SKEW}_{PARTIES}clients_intervals.txt", "r") as reader:
            for i in range(PARTIES):
                pass# print(reader.readline())
    except:
        print("File does not exist")
    
    # Get fedavg intervals search
    try:
        with open(f"{experiment_directory}intervals/intervals_{DATASET.lower()}_feature_skew_{SKEW}_{PARTIES}clients.txt", "r") as reader:
            print("\nFederated average best intervals\n")

            for i in range(1):
                pass #print(reader.readline())

    except:
        print("File does not exist")
    
DATASET= "EMNIST" #["MNIST", "SVHN_CROPPED", "CIFAR10"]
SKEW="0.02" #["0.02",0.1"] for feature skew
PARTIES=10 #["10", "20"]
#get_res(DATASET, SKEW, PARTIES)
for p in [10, 20]:
    for s in ["0.02","0.1"]:
        get_res(DATASET, s, p)

Federated average best parameters

(DeviceArray(0.85372627, dtype=float32), {'act_fn': <jax._src.custom_derivatives.custom_jvp object at 0x7f500c252f40>, 'client_lr': 0.3, 'server_lr': 0.03162277660168379, 'client_momentum': 0.0, 'server_momentum': 0.9, 'batch_size': 32, 'epochs_per_round': 2, 'rounds': 30, 'skew': 0.02})

{'client_lr': 0.021282378184795378, 'client_momentum': 0.5996034584939482, 'batch_size': 17.045599555969236}

Decentralized intervals search

File does not exist
Federated average best parameters

(DeviceArray(0.85426784, dtype=float32), {'act_fn': <jax._src.custom_derivatives.custom_jvp object at 0x7f41d4181bb0>, 'client_lr': 0.3, 'server_lr': 0.03162277660168379, 'client_momentum': 0.0, 'server_momentum': 0.9, 'batch_size': 32, 'epochs_per_round': 2, 'rounds': 30, 'skew': 0.1})

{'client_lr': 0.023989479702711106, 'client_momentum': 0.5963579489290715, 'batch_size': 18.32828493118286}

Decentralized intervals search

File does not exist
File does not exist
{'client